In [165]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import silhouette_score

In [83]:
#utilizamos el df de limpieza de jorge
df = pd.read_csv(r"C:\Users\franp\Downloads\limpieza Jorge 040325\alquileres_completo_limpio.csv")

In [84]:
#queremos saber los % de NaN de cada variable
nan_percen = df.isnull().mean() * 100
print(f'El df tiene un total de {nan_percen[nan_percen > 0].count()} variables con valores nulos')
print(f'Las variables con NaN son: {nan_percen[nan_percen > 0]} variables con valores nulos')

El df tiene un total de 13 variables con valores nulos
Las variables con NaN son: codigo_postal            35.329668
antiguedad               48.636546
conservacion             43.736120
superficie                0.262133
superficie_util           0.262133
superficie_construida     0.262133
superficie_solar         94.629920
habitaciones              1.944151
baños                     1.055812
tipo_suelo               65.842648
carpinteria_interior     90.668803
carpinteria_exterior     79.815779
precio_m2                 0.141989
dtype: float64 variables con valores nulos


In [85]:
#cp más común en Andalucía
andalucia = df[df['comunidad_autonoma'] == 'Andalucia']
most_common_cp = andalucia['codigo_postal'].mode()[0]
most_common_province = andalucia[andalucia['codigo_postal'] == most_common_cp]['provincia'].mode()[0]
print(f'El código postal más común en Andalucía es: {most_common_cp} y la provincia más común es: {most_common_province}')

El código postal más común en Andalucía es: 29602.0 y la provincia más común es: Malaga


In [ ]:
print(f'Las columnas numéricas son: {df.select_dtypes(include=['float64']).columns}')
print(f'Las columnas categóricas son: {df.select_dtypes(include=["object"]).columns}')

In [89]:
#para saber NaN por var numéricas
df_nan_num=df.select_dtypes(include=['float64']).isna().mean() * 100
df_nan_num_borrar = df_nan_num[df_nan_num > 30]
print(f'Las variables a eliminar son: {df_nan_num_borrar}')

Las variables a eliminar son: codigo_postal       35.329668
superficie_solar    94.629920
dtype: float64


In [104]:
#para saber NaN por var categoricas
df_nan_cat=df.select_dtypes(include=['O']).isna().mean() * 100
df_nan_cat_borrar = df_nan_cat[df_nan_cat > 30]
print(f'Las variables a eliminar son: {df_nan_cat_borrar}')

Las variables a eliminar son: antiguedad              48.636546
conservacion            43.736120
tipo_suelo              65.842648
carpinteria_interior    90.668803
carpinteria_exterior    79.815779
dtype: float64


In [148]:
#borramos variables num y cat del df
df2 = df.drop(columns=['identificador', 'superficie_solar', 'superficie_util', 'superficie_construida', 'codigo_postal', 'antiguedad', 'conservacion', 'tipo_suelo', 'carpinteria_interior', 'carpinteria_exterior'], inplace=False)

In [149]:
#tras eliminar las variables con NaN > 30%, vemos cuales quedan
df2_menos_30_nan = df2.isnull().mean() * 100

print(df2_menos_30_nan[df2_menos_30_nan > 0])

superficie      0.262133
habitaciones    1.944151
baños           1.055812
precio_m2       0.141989
dtype: float64


In [150]:
#seleccionamos las columnas a imputar

columns_to_imput = ['superficie', 'habitaciones', 'baños', 'precio_m2']

#crear el imputador con estrategia de la media

imputer = SimpleImputer(strategy='mean')

#imputar los valores faltantes

df2[columns_to_imput] = imputer.fit_transform(df2[columns_to_imput])

In [ ]:
cat_cols = df2.select_dtypes(include=['object']).columns

In [157]:
#convertimos el df en un array de numpy
X = df2.values

#obtener los índices de las columnas categóricas
cat_index = [df2.columns.get_loc(col) for col in cat_cols]
print('Índices de columnas categóricas:', cat_index)

Índices de columnas categóricas: [0, 1, 2, 3, 4, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43, 44, 45]


In [172]:
df2.shape[1]

47

In [ ]:
#INTENTO 1 K.PROTOTYPES

#ajustamos el modelo K-Prototypes

kproto = KPrototypes(n_clusters=5, init='Cao', random_state=42)

#ajustar el modelo a los datos

clusters = kproto.fit_predict(X, categorical=cat_index)

#añadir los clusters al df original
df2['cluster'] = clusters

print(df2.head(20))

In [169]:
X_df_num = df2.select_dtypes(include='float64').values

silhouette = silhouette_score(X_df_num, df2['cluster'])
print(f'Silhouette Score: {silhouette:.3f}')

Silhouette Score: 0.564


In [161]:
print(df2['cluster'].value_counts())

cluster
0    14390
4     8637
3     3240
1      929
2      271
Name: count, dtype: int64


In [ ]:
#INTENTO 2 - K PROTOTYPES

#ajustamos el modelo K-Prototypes

kproto = KPrototypes(n_clusters=3, init='Cao', random_state=42)

#ajustar el modelo a los datos

clusters = kproto.fit_predict(X, categorical=cat_index)

#añadir los clusters al df original
df2['cluster'] = clusters

print(df2.head(20))

                                         nombre provincia comunidad_autonoma  \
0   Apartamento en Carretera de Ciudad de Cádiz   Almeria          Andalucia   
1                            Piso en calle Vera   Almeria          Andalucia   
2        Piso en Ciudad Jardín-Tagarete-Zapillo   Almeria          Andalucia   
3                              Piso en La Envía   Almeria          Andalucia   
4          Piso en Carretera de Ciudad de Cádiz   Almeria          Andalucia   
5        Apartamento en calle Islas Canarias, 1   Almeria          Andalucia   
6              Piso en calle Ciudad de Alicante   Almeria          Andalucia   
7               Casa en calle de la Ensenada, 8   Almeria          Andalucia   
8                     Apartamento en Vera Playa   Almeria          Andalucia   
9             Apartamento en calle de Edimburgo   Almeria          Andalucia   
10                    Piso en Campillo del Moro   Almeria          Andalucia   
11                   Piso en Avda. Medit

In [171]:
X_df_num = df2.select_dtypes(include='float64').values

silhouette = silhouette_score(X_df_num, df2['cluster'])
print(f'Silhouette Score: {silhouette:.3f}')

Silhouette Score: 0.694
